# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 3:35PM,238707,20,SSNA-2173648876,Sartorius Stedim North America,Released
1,Jun 2 2022 3:29PM,238705,10,0085737147,ISDIN Corporation,Released
2,Jun 2 2022 3:17PM,238698,10,MET210869,"Methapharm, Inc.",Released
3,Jun 2 2022 3:17PM,238698,10,MET210870,"Methapharm, Inc.",Released
4,Jun 2 2022 3:17PM,238698,10,MET210871,"Methapharm, Inc.",Released
5,Jun 2 2022 3:17PM,238698,10,MET210873,"Methapharm, Inc.",Released
6,Jun 2 2022 3:17PM,238698,10,MET210874,"Methapharm, Inc.",Released
7,Jun 2 2022 3:17PM,238698,10,MET210875,"Methapharm, Inc.",Released
8,Jun 2 2022 3:17PM,238698,10,MET210866,"Methapharm, Inc.",Released
9,Jun 2 2022 3:17PM,238697,19,DEX0006920,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,238696,Released,3
36,238697,Released,11
37,238698,Released,7
38,238705,Released,1
39,238707,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238696,NaN,NaN,3.0
238697,NaN,NaN,11.0
238698,NaN,NaN,7.0
238705,NaN,NaN,1.0
238707,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,10.0,3.0,1.0
238630,5.0,8.0,0.0
238634,0.0,1.0,0.0
238643,0.0,0.0,17.0
238644,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238578,10,3,1
238630,5,8,0
238634,0,1,0
238643,0,0,17
238644,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,10,3,1
1,238630,5,8,0
2,238634,0,1,0
3,238643,0,0,17
4,238644,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238578,10,3,1
1,238630,5,8,
2,238634,,1,
3,238643,,,17
4,238644,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc."
9,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
20,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc."
23,Jun 2 2022 3:04PM,238691,10,Yusen – 3D Matrix
24,Jun 2 2022 2:59PM,238690,10,"Methapharm, Inc."
25,Jun 2 2022 2:51PM,238689,10,Eywa Pharma Inc.
26,Jun 2 2022 2:34PM,238688,20,"Exact-Rx, Inc."
27,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America,,,1
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation,,,1
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc.",,,7
3,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,11
4,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc.",,,3
5,Jun 2 2022 3:04PM,238691,10,Yusen – 3D Matrix,,,1
6,Jun 2 2022 2:59PM,238690,10,"Methapharm, Inc.",,,1
7,Jun 2 2022 2:51PM,238689,10,Eywa Pharma Inc.,,,1
8,Jun 2 2022 2:34PM,238688,20,"Exact-Rx, Inc.",,,1
9,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America,1,,
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation,1,,
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc.",7,,
3,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11,,
4,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc.",3,,
5,Jun 2 2022 3:04PM,238691,10,Yusen – 3D Matrix,1,,
6,Jun 2 2022 2:59PM,238690,10,"Methapharm, Inc.",1,,
7,Jun 2 2022 2:51PM,238689,10,Eywa Pharma Inc.,1,,
8,Jun 2 2022 2:34PM,238688,20,"Exact-Rx, Inc.",1,,
9,Jun 2 2022 2:26PM,238687,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America,1,,
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation,1,,
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc.",7,,
3,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11,,
4,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation,1.0,NaN,NaN
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc.",7.0,NaN,NaN
3,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11.0,NaN,NaN
4,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 3:35PM,238707,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 2 2022 3:29PM,238705,10,ISDIN Corporation,1.0,0.0,0.0
2,Jun 2 2022 3:17PM,238698,10,"Methapharm, Inc.",7.0,0.0,0.0
3,Jun 2 2022 3:17PM,238697,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11.0,0.0,0.0
4,Jun 2 2022 3:15PM,238696,10,"Methapharm, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3580045,58.0,25.0,14.0
12,715948,8.0,2.0,0.0
15,238630,0.0,8.0,5.0
16,715969,3.0,0.0,0.0
19,715992,12.0,10.0,0.0
20,954746,18.0,15.0,0.0
22,716058,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3580045,58.0,25.0,14.0
1,12,715948,8.0,2.0,0.0
2,15,238630,0.0,8.0,5.0
3,16,715969,3.0,0.0,0.0
4,19,715992,12.0,10.0,0.0
5,20,954746,18.0,15.0,0.0
6,22,716058,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,58.0,25.0,14.0
1,12,8.0,2.0,0.0
2,15,0.0,8.0,5.0
3,16,3.0,0.0,0.0
4,19,12.0,10.0,0.0
5,20,18.0,15.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,58.0
1,12,Released,8.0
2,15,Released,0.0
3,16,Released,3.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,14.0,0.0,5.0,0.0,0.0,0.0,0.0
Executing,25.0,2.0,8.0,0.0,10.0,15.0,0.0
Released,58.0,8.0,0.0,3.0,12.0,18.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,14.0,0.0,5.0,0.0,0.0,0.0,0.0
1,Executing,25.0,2.0,8.0,0.0,10.0,15.0,0.0
2,Released,58.0,8.0,0.0,3.0,12.0,18.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,14.0,0.0,5.0,0.0,0.0,0.0,0.0
1,Executing,25.0,2.0,8.0,0.0,10.0,15.0,0.0
2,Released,58.0,8.0,0.0,3.0,12.0,18.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()